In [1]:
import numpy as np
import pandas as pd
import scipy
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv(os.path.join('.', 'quantity_by_cluster.csv'))
df = df.drop(columns='Unnamed: 0')
df = df.astype({'cluster_a':'category',
               'edad':'category',
               'sexo':'category'})
df

,cluster_a,edad,sexo,poblacion,grupo_edad,escolarizacion,atencion_salud_formal,trabajo_remunerado,p_escolarizacion,p_atencion_salud_formal,...,p_emparejado,p_no_emparejado,vivia_un_anio,vivia_cinco_anio,un_hijo,dos_hijos,tres_o_mas_hijos,sin_hijos,emparejado,no_emparejado
0,0,7,m,56619,5,55425,6589,17045,0.978912,0.830476,...,0.666119,0.326904,2301,7203,43580,20581,6779,22,37715,18509
1,0,8,h,56354,6,54260,5096,41069,0.962842,0.767701,...,0.677964,0.313784,2009,6088,0,0,0,0,38206,17683
2,0,8,m,55959,6,54382,7408,17459,0.971819,0.833671,...,0.688075,0.305706,1769,5718,41879,24296,11519,14,38504,17107
3,0,9,h,50406,6,47997,5142,36933,0.952208,0.771377,...,0.701047,0.291751,1443,4609,0,0,0,0,35337,14706
4,0,9,m,50317,6,48342,7522,15133,0.960749,0.832724,...,0.693781,0.299819,1218,4028,35036,24195,15182,15,34909,15086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,4,19,m,14399,7,9035,2778,1631,0.627474,0.927546,...,0.113688,0.878325,152,576,3572,4096,15228,3,1637,12647
206,4,20,h,2801,7,1707,486,633,0.609425,0.896679,...,0.377008,0.612995,39,127,0,0,0,0,1056,1717
207,4,20,m,4413,7,2680,773,471,0.607297,0.919144,...,0.119873,0.872876,49,184,1152,1265,4444,1,529,3852
208,4,21,h,1322,7,993,117,667,0.751135,0.860294,...,0.487897,0.488654,46,132,0,0,0,0,645,646


In [3]:
df.columns

Index(['cluster_a', 'edad', 'sexo', 'poblacion', 'grupo_edad',
       'escolarizacion', 'atencion_salud_formal', 'trabajo_remunerado',
       'p_escolarizacion', 'p_atencion_salud_formal', 'p_trabajo_remunerado',
       'p_vivia_un_anio', 'p_vivia_cinco_anios', 'p_un_hijo', 'p_dos_hijos',
       'p_tres_o_mas_hijos', 'p_sin_hijos', 'p_emparejado', 'p_no_emparejado',
       'vivia_un_anio', 'vivia_cinco_anio', 'un_hijo', 'dos_hijos',
       'tres_o_mas_hijos', 'sin_hijos', 'emparejado', 'no_emparejado'],
      dtype='object')

In [4]:
df[df['edad']==3]['p_escolarizacion']

33     0.983854
34     0.988911
47     0.993551
48     0.995707
88     0.949488
89     0.951440
128    0.993933
129    0.995795
174    0.990704
175    0.993530
Name: p_escolarizacion, dtype: float64

## Idea principal
Se busca tomar una infografía base e irla filtrando hasta el punto que teniendo en cuenta las mismas variables se denotan diferencias de sexo en la cantidad de personas.

## Escena 1
Municipios se dividen en clusters de manera que se confunda la variable socio-económica.

In [5]:
# Aquí irían los clusters, se dividieron entre cabeceras y no cabeceras en este caso
iterator = df['cluster_a'].unique()
iterator

[0, 1, 2, 3, 4]
Categories (5, int64): [0, 1, 2, 3, 4]

# Orden de clusters, 3 - 4 - 0 - 2 - 1

## Escena 2:
Nacen 2 parejas de niños (M/F) en municipios escogidos dentro de los clusters. Análisis de edad 1 y 2 (0 a 9) en columna "enfermo", se espera que el valor sea distinto a "Nada". (Edad 1 y 2 no poseen información de trabajo)

No tiene significancia alguna, no se va a mostrar en la ppt. Se sugiere que en el proximo censo se tenga mas datos para educación

In [6]:
for cluster in iterator:
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'].isin([1, 2])) & (df['cluster_a'] == cluster)].copy()
    df_slice['tot_enfermos'] = np.round(df_slice['atencion_salud_formal'] / df_slice['p_atencion_salud_formal'])
    df_slice['no_atencion_salud_formal'] = df_slice['tot_enfermos'] - df_slice['atencion_salud_formal']
    porcentaje_atend_h = df_slice[df_slice['sexo'] == 'h']['atencion_salud_formal'].sum() / df_slice[df_slice['sexo'] == 'h']['tot_enfermos'].sum()
    porcentaje_atend_m = df_slice[df_slice['sexo'] == 'm']['atencion_salud_formal'].sum() / df_slice[df_slice['sexo'] == 'm']['tot_enfermos'].sum()
    print('La cantidad de niños edades 0-9 que reciben atención médica es {0:.2f}%'.format(porcentaje_atend_h * 100))
    print('La cantidad de niñas edades 0-9 que reciben atención médica es {0:.2f}%'.format(porcentaje_atend_m * 100))
    # Estadística
    crosstab_counts = df_slice.groupby('sexo')[['no_atencion_salud_formal', 'atencion_salud_formal']].sum()
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Atención médica {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 0
La cantidad de niños edades 0-9 que reciben atención médica es 81.40%
La cantidad de niñas edades 0-9 que reciben atención médica es 81.20%
Atención médica es independiente de sexo

Para el cluster 1
La cantidad de niños edades 0-9 que reciben atención médica es 79.29%
La cantidad de niñas edades 0-9 que reciben atención médica es 78.51%
Atención médica no es independiente de sexo

Para el cluster 2
La cantidad de niños edades 0-9 que reciben atención médica es 75.33%
La cantidad de niñas edades 0-9 que reciben atención médica es 74.63%
Atención médica es independiente de sexo

Para el cluster 3
La cantidad de niños edades 0-9 que reciben atención médica es 86.25%
La cantidad de niñas edades 0-9 que reciben atención médica es 85.45%
Atención médica no es independiente de sexo

Para el cluster 4
La cantidad de niños edades 0-9 que reciben atención médica es 81.51%
La cantidad de niñas edades 0-9 que reciben atención médica es 80.53%
Atención médica no es independiente 

## Escena 3: 
Edad 3 (10 a 14) se analiza columna "trabajo", se espera tenga el valor de "Estudio"

In [7]:
for cluster in iterator:
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'] == 3) & (df['cluster_a'] == cluster)].copy()
    df_slice['no_escolarizacion'] = df_slice['poblacion'] - df_slice['escolarizacion']
    print('La cantidad de jovenes hombres edad 3 no escolarizados es {0:.2f}%'.
          format((1 - df_slice[df_slice['sexo'] == 'h']['p_escolarizacion']).values[0] * 100))
    print('La cantidad de jovenes mujeres edad 3 no escolarizados es {0:.2f}%'.
          format((1 - df_slice[df_slice['sexo'] == 'm']['p_escolarizacion']).values[0] * 100))
    # Estadística
    crosstab_counts = df_slice.groupby('sexo')[['escolarizacion', 'no_escolarizacion']].sum()
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Estudio {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 0
La cantidad de jovenes hombres edad 3 no escolarizados es 0.64%
La cantidad de jovenes mujeres edad 3 no escolarizados es 0.43%
Estudio no es independiente de sexo

Para el cluster 1
La cantidad de jovenes hombres edad 3 no escolarizados es 1.61%
La cantidad de jovenes mujeres edad 3 no escolarizados es 1.11%
Estudio no es independiente de sexo

Para el cluster 2
La cantidad de jovenes hombres edad 3 no escolarizados es 5.05%
La cantidad de jovenes mujeres edad 3 no escolarizados es 4.86%
Estudio es independiente de sexo

Para el cluster 3
La cantidad de jovenes hombres edad 3 no escolarizados es 0.61%
La cantidad de jovenes mujeres edad 3 no escolarizados es 0.42%
Estudio no es independiente de sexo

Para el cluster 4
La cantidad de jovenes hombres edad 3 no escolarizados es 0.93%
La cantidad de jovenes mujeres edad 3 no escolarizados es 0.65%
Estudio no es independiente de sexo



## Escena 4: (Query)
Edad 3 (10 a 14) se espera que no tengan hijos.

Query: Obtener datos para 0, 1, 2 y 3+ hijos junto a variables de escolarizacion para compararlos

In [8]:
for cluster in iterator:
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'] == 3) & (df['cluster_a'] == cluster) & (df['sexo'] == 'm')].copy()
    print('La cantidad de jovenes edad 3 sin hijos es {0:.2f}%'.format((df_slice['p_sin_hijos'].values[0] * 100)))
    print('La cantidad de jovenes edad 3 con un hijo es {0:.2f}%'.format((df_slice['p_un_hijo'].values[0] * 100)))
    print('La cantidad de jovenes edad 3 con dos hijos es {0:.2f}%'.format((df_slice['p_dos_hijos'].values[0] * 100)))
    print('La cantidad de jovenes edad 3 con tres o mas hijos es {0:.2f}%\n'.format((df_slice['p_tres_o_mas_hijos'].values[0] * 100)))

Para el cluster 0
La cantidad de jovenes edad 3 sin hijos es 0.10%
La cantidad de jovenes edad 3 con un hijo es 0.27%
La cantidad de jovenes edad 3 con dos hijos es 0.03%
La cantidad de jovenes edad 3 con tres o mas hijos es 0.20%

Para el cluster 1
La cantidad de jovenes edad 3 sin hijos es 0.29%
La cantidad de jovenes edad 3 con un hijo es 0.44%
La cantidad de jovenes edad 3 con dos hijos es 0.06%
La cantidad de jovenes edad 3 con tres o mas hijos es 0.61%

Para el cluster 2
La cantidad de jovenes edad 3 sin hijos es 0.79%
La cantidad de jovenes edad 3 con un hijo es 0.73%
La cantidad de jovenes edad 3 con dos hijos es 0.16%
La cantidad de jovenes edad 3 con tres o mas hijos es 1.70%

Para el cluster 3
La cantidad de jovenes edad 3 sin hijos es 0.10%
La cantidad de jovenes edad 3 con un hijo es 0.16%
La cantidad de jovenes edad 3 con dos hijos es 0.02%
La cantidad de jovenes edad 3 con tres o mas hijos es 0.21%

Para el cluster 4
La cantidad de jovenes edad 3 sin hijos es 0.12%
La ca

## Escena 5: (Falta en tabla quantity)
Edad 4 (15 a 19) los jovenes que ya mantienen su hogar deberían ser alfabetas para poder trabajar y mantenerse dignamente, se tiene encuenta "parentesco", se espera que si tiene el valor "Jefe_Hogar" o "Pareja" entonces "alfabeta" sea "True". ¿Se filtra población?

Tabla: Falta cantidad de parentescos ("Jefe_Hogar" o "Pareja") en tabla quantity_by_cluster

## Escena 6
Edad 5 (25 a 39) luego de haber logrado el camino de la educación, de los no hijos y de la salud, ahora se evalúan las condiciones de vida. Empezamos por analizar cuántos no poseen remuneración.

In [9]:
for cluster in iterator:
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'].isin([5, 6, 7])) & (df['cluster_a'] == cluster)].copy()
    df_slice['no_rem'] = df_slice['poblacion'] - df_slice['trabajo_remunerado']
    p_rem_h = df_slice[df_slice['sexo'] == 'h']['trabajo_remunerado'].sum() / df_slice[df_slice['sexo'] == 'h']['poblacion'].sum()
    p_rem_m = df_slice[df_slice['sexo'] == 'm']['trabajo_remunerado'].sum() / df_slice[df_slice['sexo'] == 'm']['poblacion'].sum()
    print('La cantidad de hombres de edad 25-40 que tienen trabajo remunerado es {0:.2f}%'.format(p_rem_h * 100))
    print('La cantidad de mujeres de edad 25-40 que tienen trabajo remunerado es {0:.2f}%'.format(p_rem_m * 100))
    # Estadística
    crosstab_counts = df_slice.groupby('sexo')[['trabajo_remunerado', 'no_rem']].sum()
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Remuneración {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 0
La cantidad de hombres de edad 25-40 que tienen trabajo remunerado es 63.76%
La cantidad de mujeres de edad 25-40 que tienen trabajo remunerado es 26.98%
Remuneración no es independiente de sexo

Para el cluster 1
La cantidad de hombres de edad 25-40 que tienen trabajo remunerado es 66.03%
La cantidad de mujeres de edad 25-40 que tienen trabajo remunerado es 24.77%
Remuneración no es independiente de sexo

Para el cluster 2
La cantidad de hombres de edad 25-40 que tienen trabajo remunerado es 42.55%
La cantidad de mujeres de edad 25-40 que tienen trabajo remunerado es 20.49%
Remuneración no es independiente de sexo

Para el cluster 3
La cantidad de hombres de edad 25-40 que tienen trabajo remunerado es 72.06%
La cantidad de mujeres de edad 25-40 que tienen trabajo remunerado es 56.64%
Remuneración no es independiente de sexo

Para el cluster 4
La cantidad de hombres de edad 25-40 que tienen trabajo remunerado es 68.76%
La cantidad de mujeres de edad 25-40 que tienen t

## Escena 7
Edad 5 (25 a 39) Análisis migratorio

In [10]:
for cluster in iterator:
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'].isin([5, 6, 7])) & (df['cluster_a'] == cluster)].copy()
    df_slice['no_v5'] = df_slice['poblacion'] - df_slice['vivia_cinco_anio']
    p_rem_h = df_slice[df_slice['sexo'] == 'h']['vivia_cinco_anio'].sum() / df_slice[df_slice['sexo'] == 'h']['poblacion'].sum()
    p_rem_m = df_slice[df_slice['sexo'] == 'm']['trabajo_remunerado'].sum() / df_slice[df_slice['sexo'] == 'm']['poblacion'].sum()
    print('La cantidad de hombres de edad 25-40 que debieron inmigrar es {0:.2f}%'.format(p_rem_h * 100))
    print('La cantidad de mujeres de edad 25-40 que debieron inmigrar es {0:.2f}%'.format(p_rem_m * 100))
    # Estadística
    crosstab_counts = df_slice.groupby('sexo')[['vivia_cinco_anio', 'no_v5']].sum()
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Inmigracion {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 0
La cantidad de hombres de edad 25-40 que debieron inmigrar es 11.95%
La cantidad de mujeres de edad 25-40 que debieron inmigrar es 26.98%
Inmigracion no es independiente de sexo

Para el cluster 1
La cantidad de hombres de edad 25-40 que debieron inmigrar es 10.71%
La cantidad de mujeres de edad 25-40 que debieron inmigrar es 24.77%
Inmigracion no es independiente de sexo

Para el cluster 2
La cantidad de hombres de edad 25-40 que debieron inmigrar es 5.41%
La cantidad de mujeres de edad 25-40 que debieron inmigrar es 20.49%
Inmigracion no es independiente de sexo

Para el cluster 3
La cantidad de hombres de edad 25-40 que debieron inmigrar es 14.60%
La cantidad de mujeres de edad 25-40 que debieron inmigrar es 56.64%
Inmigracion no es independiente de sexo

Para el cluster 4
La cantidad de hombres de edad 25-40 que debieron inmigrar es 12.94%
La cantidad de mujeres de edad 25-40 que debieron inmigrar es 36.88%
Inmigracion no es independiente de sexo



## Escena 8 (Query)
Edad 5 (40 a 59) Query: Agrupar por estado civil, remunerado y obtener cantidades

In [11]:
for cluster in iterator:
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'].isin([8, 9, 10, 11])) & (df['cluster_a'] == cluster)].copy()
    p_emp_h = df_slice[df_slice['sexo'] == 'h']['emparejado'].sum() / df_slice[df_slice['sexo'] == 'h']['poblacion'].sum()
    p_emp_m = df_slice[df_slice['sexo'] == 'm']['emparejado'].sum() / df_slice[df_slice['sexo'] == 'm']['poblacion'].sum()
    print('La cantidad de hombres de edad 40-59 emparejados es {0:.2f}%'.format(p_emp_h * 100))
    print('La cantidad de mujeres de edad 40-59 emparejadas es {0:.2f}%'.format(p_emp_m * 100))
    # Estadística
    crosstab_counts = df_slice.groupby('sexo')[['emparejado', 'no_emparejado']].sum()
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Remuneración {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 0
La cantidad de hombres de edad 40-59 emparejados es 70.05%
La cantidad de mujeres de edad 40-59 emparejadas es 68.56%
Remuneración no es independiente de sexo

Para el cluster 1
La cantidad de hombres de edad 40-59 emparejados es 75.48%
La cantidad de mujeres de edad 40-59 emparejadas es 71.57%
Remuneración no es independiente de sexo

Para el cluster 2
La cantidad de hombres de edad 40-59 emparejados es 74.23%
La cantidad de mujeres de edad 40-59 emparejadas es 71.55%
Remuneración no es independiente de sexo

Para el cluster 3
La cantidad de hombres de edad 40-59 emparejados es 68.83%
La cantidad de mujeres de edad 40-59 emparejadas es 60.69%
Remuneración no es independiente de sexo

Para el cluster 4
La cantidad de hombres de edad 40-59 emparejados es 73.30%
La cantidad de mujeres de edad 40-59 emparejadas es 66.35%
Remuneración no es independiente de sexo



## Escena 9 (Query)
Edades 40-60. Query: Filtrar por remunerado y ver la cantidad de nivel educativo por sexo.